In [1]:
import re
import json

import pandas as pd

In [2]:
wiki_path = '../data/wikipedia_documents.json'
with open(wiki_path, 'r') as file:
    wiki_doc = json.load(file)

print(f'wiki_doc 길이 : {len(wiki_doc)}')

wiki_doc 길이 : 60613


In [21]:
# 한글 비율을 계산하는 함수
def korean_ratio(text):
    """주어진 텍스트에서 한글 문자의 비율을 계산합니다."""
    korean_chars = re.findall(r'[가-힣]', text)
    total_chars = len(text)
    return len(korean_chars) / total_chars if total_chars > 0 else 0.0

# 만약 max_len을 넘을 때 동작
def split_text_by_max_len(text, max_len=5000, cut_text_based_enter=True):    
    # 문장 구분 기준 설정 (\n을 기준으로 할지, .?!을 기준으로 할지)
    if cut_text_based_enter and '\n' in text:
        sentences = text.split('\n')
    else:
        sentences = re.split(r'(?<=[.!?])\s+', text)
    
    split_texts = []
    current_text = []

    # 문장을 추가하면서 길이가 max_len을 넘는 경우 나눔
    for sentence in sentences:
        sentence = sentence.strip()  # 공백 제거
        if not sentence:  # 빈 문자열은 건너뜀
            continue

        if len(' '.join(current_text)) + len(sentence) + 1 <= max_len:
            current_text.append(sentence)
        else:
            split_texts.append(' '.join(current_text).strip())
            current_text = [sentence]
    
    # 마지막으로 남은 문장 처리
    if current_text:
        split_texts.append(' '.join(current_text).strip())
    
    return split_texts # list type output


def preprocess_json(wiki_doc, without_sohak=True, max_len=5000, cut_text_based_enter=True, ratio=0.5, save=None):
    
    # wiki json file을 dataframe으로
    df = pd.DataFrame(wiki_doc).transpose()
    
    # 소학언해 제외
    if without_sohak:
        df = df[df['title']!='소학언해']
    
    # ratio 비율 이상의 df의 각 열에 기준 길이보다 긴 경우         
    new_rows = []
    
    for idx, row in df.iterrows():
        text = row['text']
        
        # 기준 길이(max_len) 보다 긴 경우
        if len(text) > max_len:
            split_texts = split_text_by_max_len(text, max_len, cut_text_based_enter)
            
            # 나뉜 텍스트 각각에 대해 새로운 행을 추가
            for split_text in split_texts:
                new_row = row.copy()  # 기존 행을 복사
                new_row['text'] = split_text  # 텍스트만 나눠진 것으로 변경
                new_rows.append(new_row)  # 새로운 행을 리스트에 추가
        else:
            new_rows.append(row)  # max_len 이하인 경우 원래 행 추가
    
    # 나눠진 행들로 새로운 DataFrame 생성
    new_df = pd.DataFrame(new_rows)
    
    # 각 텍스트의 한글 비율을 계산
    new_df['korean_ratio'] = new_df['text'].apply(korean_ratio)    
    new_df = new_df[new_df['korean_ratio'] > ratio] # 한글 포함 비율 기준으로 필터링
    new_df = new_df.drop(columns=['korean_ratio'])
    
    """
    ### 데이터 저장 부분 ###
    """
    # save 인자가 주어지면 파일 저장
    if save:
        new_df_transposed = new_df.transpose()  # 원래 형태로 변환
        json_data = new_df_transposed.to_dict()

        # 파일 저장
        with open(f'{save}.json', 'w', encoding='utf-8') as file:
            json.dump(json_data, file, ensure_ascii=False, indent=4)
    return new_df

In [35]:
file_name = 'wiki_60'
ratio = 0.6
preprocess_json(wiki_doc, 
                without_sohak=True, 
                max_len=5000, 
                cut_text_based_enter=True,  # '\n'이 text 내에 포함되어 있으면 
                ratio=ratio,
                save=file_name)

/tmp/ipykernel_804968/2307319244.py:79: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_data = new_df_transposed.to_dict()


,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4
...,...,...,...,...,...,...,...,...
60606,"곰 자체는 페름 지역 숲에 사는 동물이 풍부함을 상징하며, 은색인 이유는 금속, 소...",위키피디아,None,None,페름의 문장,None,None,60606
60607,"1967년 소련 통치기 페름 시 의회는 새 문장 공모전을 개최하였으며, 1968년까...",위키피디아,None,None,페름의 문장,None,None,60607
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610


<br><br>

---

<br><br>

** Output 확인 **

In [43]:
def confirm_output(wiki_doc, file_name):
    # wiki_doc을 DataFrame으로 변환
    wiki_df = pd.DataFrame(wiki_doc).transpose()

    output = f'{file_name}.json'
    with open(output, 'r') as file:
        output_json = json.load(file)

    # 출력 JSON을 DataFrame으로 변환
    out_df = pd.DataFrame(output_json).transpose()
    
    # 컬럼 비교
    if wiki_df.columns.equals(out_df.columns):
        print('Wiki 문서와 출력 문서의 구성이 같습니다.')
    else:
        print('Wiki 문서와 출력 문서의 구성이 다릅니다.')
        print(f'wiki : {wiki_df.columns.tolist()}')
        print(f'{file_name} : {out_df.columns.tolist()}')
        
    # 각 텍스트의 한글 비율을 계산
    out_df['korean_ratio'] = out_df['text'].apply(korean_ratio)    
    
    # 최저 한글 비율 출력
    min_korean_ratio = out_df['korean_ratio'].min()
    print('\n\n==========================================================\n')
    print(f'{file_name}의 최저 한글 비율은 {min_korean_ratio:.2f}입니다.')
    print(out_df[out_df['korean_ratio'] == min_korean_ratio])
    
    # 평균 한글 비율 계산 및 출력
    average_korean_ratio = out_df['korean_ratio'].mean()
    print(f'{file_name}의 평균 한글 비율은 {average_korean_ratio:.2f}입니다.')
    
    # 가장 긴 텍스트 출력
    longest_text = out_df.loc[out_df['text'].str.len().idxmax()]['text']
    average_length = out_df['text'].str.len().mean()
    
    print('\n\n==========================================================\n')
    print(f'{file_name}의 가장 긴 text는 길이가 "{len(longest_text)}"입니다.')
    print(f'{file_name}의 평균 text 길이는 "{average_length:.2f}"입니다.')
    print(out_df[out_df['text'] == longest_text])
    
    # out_df의 샘플 수 반환
    sample_count = out_df.shape[0]
    return sample_count  # 샘플 수 반환
    
# file_name = 'wiki_60'
# confirm_output(wiki_doc, file_name)    

In [45]:
file_names = [f'wiki_{i}' for i in range(10, 61, 10)]

for f in file_names:
    outlen = confirm_output(wiki_doc, f)
    print(f'\n\n\n file_name = {f} / length = {outlen}')
    print('\n\n\n============================================================================\n\n')
    

Wiki 문서와 출력 문서의 구성이 같습니다.



wiki_10의 최저 한글 비율은 0.10입니다.
                                                    text corpus_source   url  \
11254  ;수록곡\n# Rising Road~ (Main Theme)\n# ハニ-チ-ズ (H...         위키피디아  None   

      domain     title author  html document_id  korean_ratio  
11254   None  프러포즈 대작전   None  None       11254       0.10017  
wiki_10의 평균 한글 비율은 0.63입니다.



wiki_10의 가장 긴 text는 길이가 "9473"입니다.
wiki_10의 평균 text 길이는 "733.24"입니다.
                                                    text corpus_source   url  \
39726  1513년(계유년,1세) 11월 16일 출생하였는데 태어날 때부터 아름다운 기질이 ...         위키피디아  None   

      domain title author  html document_id  korean_ratio  
39726   None   정종영   None  None       39726      0.685105  



 file_name = wiki_10 / length = 60114





Wiki 문서와 출력 문서의 구성이 같습니다.



wiki_20의 최저 한글 비율은 0.20입니다.
                                                    text corpus_source   url  \
27709  콜키쿰과에 속하는 속 목록은 다음과 같다.\n \n \n* Androcymbium\...         위키피디아  None   

      dom